In [3]:
import numpy as np
from PIL import Image
import torch
from transformers import TrainingArguments, Trainer



/projects/compsci/Yue/Anaconda3/envs/torch_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import glob
import numpy as np
from datasets import Dataset, Image as ImageDs
#data_set = 'cifar10' ###epochs:10, 50, 100
#data_set = 'food101'  ###epochs: 10, 100
#data_set = 'cats_vs_dogs'  ###epochs:1000, 5000, 
#data_set = 'mnist'  ###epochs:1500, 2000

#list_of_files = glob.glob(f'{data_set}_validation_img_2stage_e{str(num_epoch)}/*.png')

data_set = 'st-net'


In [5]:
import sys
sys.path.insert(1, '/projects/li-lab/Yue/SpatialAnalysis/py/ST-net-experiments') ##~wont work, has to start with /Users
sys.path.insert(1, '/projects/li-lab/Yue/SpatialAnalysis/py') ##~wont work, has to start with /Users

import DataSTNet_TumorClassification as DataObj

#2-fold cross validation
np.random.seed(0)
random_indx = np.random.choice(len(DataObj.X), len(DataObj.X)//2)

train_image_url = np.array(DataObj.X)[random_indx]
test_image_url = np.delete(np.array(DataObj.X), random_indx)
train_voxels = np.array(DataObj.voxel_ids)[random_indx]

train_labels = np.array(DataObj.Y_filtered)[random_indx]
test_labels = np.delete(np.array(DataObj.Y_filtered),random_indx)
test_voxels = np.delete(np.array(DataObj.voxel_ids),random_indx)



from datasets import Dataset

train_ds = Dataset.from_dict({"img": train_image_url, "label":train_labels.astype(int)}).cast_column("img", ImageDs())
val_ds = Dataset.from_dict({"img": test_image_url, "label":test_labels.astype(int)}).cast_column("img", ImageDs())

print(train_ds)
print(val_ds)

          xcoord  ycoord   lab  tumor
xcoord                               
C1_10_17      10      17    L3  tumor
C1_11_17      11      17    L1  tumor
C1_12_17      12      17    L6  tumor
C1_13_17      13      17    L7  tumor
C1_14_17      14      17    L4    non
...          ...     ...   ...    ...
C1_14_34      14      34  L254    non
C1_15_34      15      34  L251    non
C1_16_34      16      34  L256    non
C1_17_34      17      34  L252    non
C1_18_34      18      34  L250    non

[256 rows x 4 columns]
10x17    tumor
11x17    tumor
12x17    tumor
13x17    tumor
14x17      non
         ...  
14x34      non
15x34      non
16x34      non
17x34      non
18x34      non
Name: tumor, Length: 256, dtype: object
          xcoord  ycoord   lab  tumor
xcoord                               
C2_4_13        4      13    L6    non
C2_5_13        5      13    L5    non
C2_6_13        6      13    L1    non
C2_7_13        7      13    L2    non
C2_15_13      15      13    L3    non
...        

          xcoord  ycoord   lab  tumor
xcoord                               
D2_9_10        9      10    L1  tumor
D2_10_11      10      11    L2  tumor
D2_9_11        9      11    L3    non
D2_12_11      12      11    L4  tumor
D2_11_12      11      12    L5  tumor
...          ...     ...   ...    ...
D2_20_34      20      34  L427  tumor
D2_24_34      24      34  L428  tumor
D2_23_34      23      34  L429  tumor
D2_18_34      18      34  L430  tumor
D2_25_34      25      34  L431  tumor

[431 rows x 4 columns]
9x10     tumor
10x11    tumor
9x11       non
12x11    tumor
11x12    tumor
         ...  
20x34    tumor
24x34    tumor
23x34    tumor
18x34    tumor
25x34    tumor
Name: tumor, Length: 431, dtype: object
          xcoord  ycoord   lab  tumor
xcoord                               
E1_17_11      17      11    L1  tumor
E1_22_11      22      11    L2  tumor
E1_8_11        8      11    L3  tumor
E1_10_11      10      11    L4  tumor
E1_9_11        9      11    L5  tumor
...        

          xcoord  ycoord   lab  tumor
xcoord                               
D2_11_9       11       9    L2    non
D2_12_9       12       9   L12    non
D2_13_9       13       9   L13    non
D2_14_9       14       9    L8    non
D2_15_9       15       9   L11    non
...          ...     ...   ...    ...
D2_12_34      12      34  L612    non
D2_13_34      13      34  L613    non
D2_14_34      14      34  L609    non
D2_15_34      15      34  L611  tumor
D2_16_34      16      34  L610  tumor

[613 rows x 4 columns]
11x9       non
12x9       non
13x9       non
14x9       non
15x9       non
         ...  
12x34      non
13x34      non
14x34      non
15x34    tumor
16x34    tumor
Name: tumor, Length: 613, dtype: object
          xcoord  ycoord   lab  tumor
xcoord                               
E1_12_10      12      10   L12    non
E1_13_10      13      10   L13    non
E1_14_10      14      10    L4    non
E1_15_10      15      10    L7    non
E1_16_10      16      10   L14    non
...        

          xcoord  ycoord   lab  tumor
xcoord                               
D2_7_4         7       4    L1  tumor
D2_6_4         6       4    L2    non
D2_7_5         7       5    L3  tumor
D2_8_5         8       5    L4  tumor
D2_6_5         6       5    L5    non
...          ...     ...   ...    ...
D2_24_32      24      32  L561    non
D2_22_33      22      33  L562    non
D2_21_33      21      33  L563    non
D2_24_33      24      33  L564    non
D2_23_33      23      33  L565    non

[565 rows x 4 columns]
7x4      tumor
6x4        non
7x5      tumor
8x5      tumor
6x5        non
         ...  
24x32      non
22x33      non
21x33      non
24x33      non
23x33      non
Name: tumor, Length: 565, dtype: object
          xcoord  ycoord   lab  tumor
xcoord                               
E1_9_4         9       4    L1  tumor
E1_11_4       11       4    L2  tumor
E1_8_4         8       4    L3  tumor
E1_10_4       10       4    L4  tumor
E1_7_4         7       4    L5  tumor
...        

          xcoord  ycoord   lab  tumor
xcoord                               
C1_11_2       11       2    L1  tumor
C1_9_2         9       2    L2  tumor
C1_6_2         6       2    L3  tumor
C1_10_2       10       2    L4  tumor
C1_12_2       12       2    L5  tumor
...          ...     ...   ...    ...
C1_25_27      25      27  L485  tumor
C1_23_27      23      27  L486  tumor
C1_24_27      24      27  L487  tumor
C1_22_28      22      28  L488  tumor
C1_23_28      23      28  L489  tumor

[489 rows x 4 columns]
11x2     tumor
9x2      tumor
6x2      tumor
10x2     tumor
12x2     tumor
         ...  
25x27    tumor
23x27    tumor
24x27    tumor
22x28    tumor
23x28    tumor
Name: tumor, Length: 489, dtype: object
          xcoord  ycoord   lab  tumor
xcoord                               
C2_3_7         3       7    L1  tumor
C2_2_7         2       7    L2  tumor
C2_6_8         6       8    L3  tumor
C2_5_8         5       8    L4  tumor
C2_3_8         3       8    L5  tumor
...        

          xcoord  ycoord   lab  tumor
xcoord                               
D1_23_8       23       8    L1  tumor
D1_24_8       24       8    L2  tumor
D1_22_9       22       9    L3  tumor
D1_21_9       21       9    L4  tumor
D1_23_9       23       9    L5  tumor
...          ...     ...   ...    ...
D1_9_28        9      28  L327  tumor
D1_8_28        8      28  L328  tumor
D1_11_29      11      29  L329  tumor
D1_9_29        9      29  L330  tumor
D1_10_29      10      29  L331  tumor

[331 rows x 4 columns]
23x8     tumor
24x8     tumor
22x9     tumor
21x9     tumor
23x9     tumor
         ...  
9x28     tumor
8x28     tumor
11x29    tumor
9x29     tumor
10x29    tumor
Name: tumor, Length: 331, dtype: object
          xcoord  ycoord   lab  tumor
xcoord                               
D2_16_10      16      10    L1  tumor
D2_17_11      17      11    L2  tumor
D2_22_11      22      11    L3  tumor
D2_21_11      21      11    L4  tumor
D2_14_11      14      11    L5  tumor
...        

In [6]:
train_voxels

array(['BC24220_E1_7x17', 'BC24105_C2_15x18', 'BC24044_E1_14x28', ...,
       'BC23508_D2_24x25', 'BC24220_E1_18x22', 'BC24044_D2_25x20'],
      dtype='<U16')

In [7]:
random_indx

array([ 2732, 10799,  9845, ..., 20675,  2995,  8798])

In [8]:
test_voxels

array(['BC23287_C1_14x27', 'BC23287_C1_15x31', 'BC23287_C1_13x27', ...,
       'BC23803_E2_21x18', 'BC23803_E2_26x14', 'BC23803_E2_17x28'],
      dtype='<U16')

In [9]:
diff_corr1 = []
diff_corr2= []

for i in range(500):
    f1 = f'{data_set}_tumorPred_train_img_Stage1/{train_voxels[i]}_train.png'
    im_array1 = np.asarray(Image.open(f1).convert('RGB'))
    
    f2 = f'{data_set}_tumorPred_validation_img_Stage2/{train_voxels[i]}_test.png'
    im_array2 = np.asarray(Image.open(f2).convert('RGB'))
    cor1 = np.corrcoef(im_array1.flatten(), im_array2.flatten())[0,1]
    
    #print(cor1)
    
    f1 = f'{data_set}_tumorPred_train_img_Stage2/{test_voxels[i]}_train.png'
    im_array1 = np.asarray(Image.open(f1).convert('RGB'))
    
    f2 = f'{data_set}_tumorPred_validation_img_Stage1/{test_voxels[i]}_test.png'
    im_array2 = np.asarray(Image.open(f2).convert('RGB'))
    cor2 = np.corrcoef(im_array1.flatten(), im_array2.flatten())[0,1]
    
    #print(cor2)
    diff_corr1.append(cor1)
    diff_corr2.append(cor2)




In [10]:
import pandas as pd
tavac_df1 = pd.DataFrame(diff_corr1)
tavac_df1['spot_id'] = train_voxels[0:500]
tavac_df1['stage'] = 'Stage1'

tavac_df1.columns = ['TAVAC','spot_id','stage']

In [11]:
import pandas as pd
tavac_df2 = pd.DataFrame(diff_corr2)
tavac_df2['spot_id'] = test_voxels[0:500]
tavac_df2['stage'] = 'Stage2'

tavac_df2.columns = ['TAVAC','spot_id','stage']

In [12]:
pd.set_option('display.max_rows', None)

tavac_df = tavac_df2.append(tavac_df1)
tavac_df.index = tavac_df['spot_id']
tavac_df = tavac_df.sort_values(by=['TAVAC'], ascending=False)

tavac_df


,TAVAC,spot_id,stage
spot_id,,,
BC23287_C2_8x19,0.995555,BC23287_C2_8x19,Stage2
BC23287_C1_13x27,0.995031,BC23287_C1_13x27,Stage2
BC23287_C2_10x17,0.994790,BC23287_C2_10x17,Stage2
BC23287_D1_16x21,0.994525,BC23287_D1_16x21,Stage2
BC24044_E1_21x28,0.992082,BC24044_E1_21x28,Stage1
BC23287_C2_5x15,0.992039,BC23287_C2_5x15,Stage2
BC23287_C1_14x27,0.991935,BC23287_C1_14x27,Stage2
BC23287_C2_13x18,0.991695,BC23287_C2_13x18,Stage2
BC23287_C1_3x31,0.991186,BC23287_C1_3x31,Stage2


In [13]:
#tavac_df.to_csv('../../output/stnet_tavac.csv')
import pandas as pd
tavac_df = pd.read_csv('../../output/stnet_tavac.csv')

In [14]:
tavac_df.loc[tavac_df.TAVAC < 0.85,:]

,spot_id,TAVAC,spot_id.1,stage
863,BC23287_D1_9x25,0.849933,BC23287_D1_9x25,Stage2
864,BC23508_E2_32x21,0.848352,BC23508_E2_32x21,Stage1
865,BC23287_D1_17x23,0.848239,BC23287_D1_17x23,Stage2
866,BC23287_D1_24x23,0.847176,BC23287_D1_24x23,Stage2
867,BC23287_C2_10x24,0.846558,BC23287_C2_10x24,Stage2
868,BC23277_E1_26x9,0.846180,BC23277_E1_26x9,Stage1
869,BC23287_D1_13x21,0.845137,BC23287_D1_13x21,Stage2
870,BC23270_E1_18x23,0.845136,BC23270_E1_18x23,Stage1
871,BC23287_C2_22x33,0.844879,BC23287_C2_22x33,Stage2
872,BC23287_D1_16x24,0.843609,BC23287_D1_16x24,Stage2


In [15]:
# diff_corr_overfit = []
# num_epoch = 100
# for i in range(500):
#     f1 = f'{data_set}_validation_img_2stage_e{str(num_epoch)}/{i}_test.png'
#     im_array1 = np.asarray(Image.open(f1).convert('RGB'))
    
#     f2 = f'{data_set}_train_img_e{str(num_epoch)}/{i}_train.png'
#     im_array2 = np.asarray(Image.open(f2).convert('RGB'))
#     cor1 = np.corrcoef(im_array1.flatten(), im_array2.flatten())[0,1]
    
#     #print(cor1)
    
#     f1 = f'{data_set}_train_img_2stage_e{str(num_epoch)}/{i}_train.png'
#     im_array1 = np.asarray(Image.open(f1).convert('RGB'))
    
#     f2 = f'{data_set}_validation_img_e{str(num_epoch)}/{i}_test.png'
#     im_array2 = np.asarray(Image.open(f2).convert('RGB'))
#     cor2 = np.corrcoef(im_array1.flatten(), im_array2.flatten())[0,1]
    
#     #print(cor2)
#     diff_corr_overfit.append(cor1)
#     diff_corr_overfit.append(cor2)


In [19]:
tavac_df.loc[tavac_df.spot_id =='BC23272_E2_22x15',:]

,spot_id,TAVAC,spot_id.1,stage
995,BC23272_E2_22x15,0.509645,BC23272_E2_22x15,Stage1


In [ ]:
# diff_corr_overfit100 = []
# num_epoch = 3000
# for i in range(500):
#     f1 = f'{data_set}_validation_img_2stage_e{str(num_epoch)}/{i}_test.png'
#     im_array1 = np.asarray(Image.open(f1).convert('RGB'))
    
#     f2 = f'{data_set}_train_img_e{str(num_epoch)}/{i}_train.png'
#     im_array2 = np.asarray(Image.open(f2).convert('RGB'))
#     cor1 = np.corrcoef(im_array1.flatten(), im_array2.flatten())[0,1]
    
#     #print(cor1)
    
#     f1 = f'{data_set}_train_img_2stage_e{str(num_epoch)}/{i}_train.png'
#     im_array1 = np.asarray(Image.open(f1).convert('RGB'))
    
#     f2 = f'{data_set}_validation_img_e{str(num_epoch)}/{i}_test.png'
#     im_array2 = np.asarray(Image.open(f2).convert('RGB'))
#     cor2 = np.corrcoef(im_array1.flatten(), im_array2.flatten())[0,1]
    
#     #print(cor2)
#     diff_corr_overfit100.append(cor1)
#     diff_corr_overfit100.append(cor2)


In [ ]:
import matplotlib.pyplot as plt
font_size = 25

fig, ax = plt.subplots()
fig.set_size_inches(3,6)
plt.hist(tavac_df['TAVAC'], alpha = 0.5)
#plt.hist(diff_corr_overfit, alpha = 0.5, label = 'epoch=100')
#plt.hist(diff_corr_overfit100, alpha = 0.5, label = 'epoch=5000')
#plt.hist(diff_corr_overfit500, alpha = 0.5, label = 'epoch=500')

#plt.legend(loc='upper left', fontsize = font_size*0.8)
plt.ylabel('Frequency',fontsize=font_size)
plt.xlabel('Correlations',fontsize=font_size)
ax.set_title(data_set + ' tumor spot\n prediction TAVAC\n',fontsize=font_size)
 
# Create names on the x-axis
#plt.xticks(x_pos, bars)
ax.tick_params(axis='x', labelsize=font_size)
ax.tick_params(axis='y', labelsize=font_size)
plt.tight_layout()

In [ ]:
import statistics
import numpy as np
def find_iqr(data):
    "Returns the interquartile range of a list of numbers"
    sorted_data = sorted(data)
    # numbers are sorted numerically
    q1 = np.percentile(sorted_data, 25)
    q3 = np.percentile(sorted_data, 75)
    iqr = q3 - q1
    print(f"q1: {q1}")
    print(f"q3: {q3}")
    return iqr
mean_corr = statistics.mean(tavac_df['TAVAC'])
median_corr = statistics.median(tavac_df['TAVAC'])
variance = np.var(tavac_df['TAVAC'])
iqr = (find_iqr(tavac_df['TAVAC']))
print(f"IQR: {iqr}")
print("Mean:", mean_corr)
print("Median:", median_corr)
print("Variance:", variance)

In [ ]:
######bad focus tavac
import glob
import numpy as np
data_set = 'cifar10'
list_of_files1 = glob.glob("BadFocusImagesBadOnly-"+data_set+"/ValidationStage1/*.png")
list_of_files2 = glob.glob("BadFocusImagesBadOnly-"+data_set+"/ValidationStage2/*.png")


diff_corr_goodfocus = []
diff_corr_badfocus = []

v_stage1 = []
for f in list_of_files1:
    ar = f.split('/')
    image_id = int(ar[-1].replace('_test.png', ''))
    v_stage1.append(image_id)
    
v_stage2 = []
for f in list_of_files2:
    ar = f.split('/')
    image_id = int(ar[-1].replace('_test.png', ''))
    v_stage2.append(image_id)


for i in range(500):


    f1 = f'{data_set}_validation_img_2stage_e{str(num_epoch)}/{i}_test.png'
    im_array1 = np.asarray(Image.open(f1).convert('RGB'))
    
    f2 = f'{data_set}_train_img_e{str(num_epoch)}/{i}_train.png'
    im_array2 = np.asarray(Image.open(f2).convert('RGB'))
    cor1 = np.corrcoef(im_array1.flatten(), im_array2.flatten())[0,1]
    
    #print(cor1)
    
    f1 = f'{data_set}_train_img_2stage_e{str(num_epoch)}/{i}_train.png'
    im_array1 = np.asarray(Image.open(f1).convert('RGB'))
    
    f2 = f'{data_set}_validation_img_e{str(num_epoch)}/{i}_test.png'
    im_array2 = np.asarray(Image.open(f2).convert('RGB'))
    cor2 = np.corrcoef(im_array1.flatten(), im_array2.flatten())[0,1]
    
    #print(cor2)

    #if (i in v_stage2) and (i in WrongPred2[0]):
    if (i in v_stage2):
        if cor1 > 0.8:
            print('vs2')
            print(i)
        diff_corr_badfocus.append(cor1)
    else:
    #elif (i not in WrongPred2[0]):
        diff_corr_goodfocus.append(cor1)
        
    #if (i in v_stage1) and (i in WrongPred1[0]):
    if (i in v_stage1):
        if cor2 > 0.8:
            print('vs1')
            print(i)
        diff_corr_badfocus.append(cor2)
    else:
    #elif (i not in WrongPred1[0]):
        diff_corr_goodfocus.append(cor2)


import matplotlib.pyplot as plt
font_size = 20

fig, ax = plt.subplots()
fig.set_size_inches(6,4)
plt.hist(diff_corr_badfocus, alpha = 0.5, label = 'bad focus')
plt.hist(diff_corr_goodfocus, alpha = 0.5, label = 'good focus')
#plt.hist(diff_corr_overfit100, alpha = 0.5, label = 'epoch=5000')
#plt.hist(diff_corr_overfit500, alpha = 0.5, label = 'epoch=500')

plt.legend(loc='upper left', fontsize = font_size*0.8)
plt.ylabel('Frequency',fontsize=font_size)
plt.xlabel('Correlations',fontsize=font_size)
ax.set_title(data_set + ' Correlation distribution vs focus\n',fontsize=font_size)
 
# Create names on the x-axis
#plt.xticks(x_pos, bars)
ax.tick_params(axis='x', labelsize=font_size)
ax.tick_params(axis='y', labelsize=font_size)
plt.tight_layout()

In [ ]:
import scipy

from scipy import stats

# Two-sample t-test
statistic, pvalue = stats.ttest_ind(diff_corr_badfocus,diff_corr_goodfocus)

print(f'Two-sample t-test: s = {statistic:5.3f}, p = {pvalue:5.5f}')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

df_bad = pd.DataFrame(diff_corr_badfocus)
df_bad['Group'] = 'Bad'

df_good = pd.DataFrame(diff_corr_goodfocus)
df_good['Group'] = 'Good'

df = df_bad.append(df_good)
df.columns = ['TAVAC','Group']

fig, ax = plt.subplots()
fig.set_size_inches(5, 5)
my_pal = {"Good": "#d1e231", "Bad": "blue"}
sns.boxplot(x="Group",y="TAVAC",data=df,ax=ax,width=0.5, showfliers = False, palette=my_pal)
#sns.violinplot(x="Group",y="TAVAC",data=df,ax=ax,width=0.5, showfliers = False, palette=my_pal)

plt.ylabel('TAVAC',fontsize=font_size)
plt.title('Comparion Good vs Bad focus',fontsize=font_size)
plt.xticks(fontsize=font_size)

In [ ]:
print("/projects/li-lab/Yue/SpatialAnalysis/py/DylanCode/BadFocusImages-"+data_set+"/ValidationStage1/*")

In [ ]:
#########prediction accuracy vs TAVAC
from datasets import load_dataset
from sklearn.metrics import accuracy_score

# imports the load_dataset function from the datasets module. Datasets
# module is a powerful tool for loading and working with various datasets
# that are commonly used in machine learning and natural language processing tasks.

data_set = 'food101'
num_epoch = 100

if data_set == 'cifar10':
    train_ds, test_ds = load_dataset(data_set, split=['train[:1000]', 'test[:100]'])
    
if data_set == 'food101':
    
# load_dataset fuction is used to load the cifar10 dataset. 
# The data set is split into two subsets: 'train' and 'test'. 

splits = train_ds.train_test_split(test_size=0.5, seed=1024)
train_ds = splits['train']
val_ds = splits['test']

from torchvision.transforms import (CenterCrop, 
                                    Compose, 
                                    Normalize, 
                                    RandomHorizontalFlip,
                                    RandomResizedCrop, 
                                    Resize, 
                                    ToTensor)

from transformers import ViTImageProcessor
# the line imports the ViTImageProcessor from the transformers library. 
# The transformers library provides state-of-the-art pretrained models and utilities
# for natural language processing and computer vision tasks 

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
image_mean, image_std = processor.image_mean, processor.image_std
size = processor.size["height"]

normalize = Normalize(mean=image_mean, std=image_std)
_train_transforms = Compose(
        [
            RandomResizedCrop(size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

_val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(size),
            ToTensor(),
            normalize,
        ]
    )

def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['img']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['img']]
    return examples

# Set the transforms
train_ds.set_transform(train_transforms)
# sets the transformation on the train_ds dataset to be train_transforms, 
# the function defined earlier

val_ds.set_transform(val_transforms)
# it sets the transformation for the validation dataset to be val_transforms
# This means that when retrieving examples from the validation dataset, 
# val_transforms functionn will be applied to preprocess the images

test_ds.set_transform(val_transforms)
# val_transfomrs function is applied to preprocess images in test dataset 


# By setting the transformations for each dataset, 
# you ensure that the appropriate preprocessing steps are automatically applied to the images
# from the respective datasets during training, validation, or testing. 
# These transformations could include resizing, cropping, flipping, and normalization, 
# as defined in the train_transforms and val_transforms functions.

import os
num_epoch = 100
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "1024"
torch.cuda.empty_cache()
#print(torch.cuda.memory_summary(device='cuda:0', abbreviated=False))
#os.environ['CUDA_VISIBLE_DEVICES']=''

if torch.cuda.is_available(): 
    dev = "cuda:0" 
else: 
    dev = "cpu"

device = torch.device(dev)
# These lines import the necessary classes, 'TrainingArguments' and 'Trainer',
# from the 'transformers' library. 
# These classes provide functionalities from configuring and executing the 
# training process for machine learning models.

metric_name = "accuracy"

# variable is set 'accuracy'
# This variable represents the name of the metric that will be used
# to evaluate performance of the model during training and determine 
# the best model based on this metric. 


from torch.utils.data import DataLoader
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return dict(accuracy=accuracy_score(predictions, labels))

args = TrainingArguments(
    f'{data_set}_checkpoints_stage2',
    save_strategy="no",
    evaluation_strategy="no",
    learning_rate=1e-3,
#     2e-5
    #0.0001
    per_device_train_batch_size=128,
#     10
    #32
    per_device_eval_batch_size=4,
    num_train_epochs=num_epoch,
    weight_decay=0,
#     it was 0.01
    #0 FOR OVERFIIY
    load_best_model_at_end=False,
    metric_for_best_model=metric_name,
    logging_dir='logs',
    remove_unused_columns=False,
    seed = 42,
    adam_beta1 = 0.9,
    adam_beta2 = 0.9,
    lr_scheduler_type = 'cosine',
    max_grad_norm = 1,
    logging_strategy = 'epoch'
)


#device = 'cpu'
model_file = "../../saved_models/ViT_pretrained_"+data_set+"_e"+str(num_epoch)+"_stage1.pt"
#model_file = "ViT_umap_Cluster_c7_C1D1_ep10k_"+region+'.h5'

model = torch.load(model_file, map_location=device)
model.eval()

import torch
# this code imports the torch library 

trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    
)

In [ ]:
outputs = trainer.predict(val_ds)


In [ ]:
outputs

In [ ]:
pred = outputs.predictions.argmax(1)

ifCorrect1 = (pred == outputs.label_ids)
WrongPred1 = np.where(~ifCorrect1)

In [ ]:
len(WrongPred1[0])/500

In [ ]:
pred

In [ ]:
outputs.label_ids

In [ ]:
val_ds[:]['label']

In [ ]:
#device = 'cpu'
model_file = "../../saved_models/ViT_pretrained_"+data_set+"_e"+str(num_epoch)+"_stage2.pt"
#model_file = "ViT_umap_Cluster_c7_C1D1_ep10k_"+region+'.h5'

model = torch.load(model_file, map_location=device)
model.eval()

import torch
# this code imports the torch library 

trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    
)
outputs = trainer.predict(train_ds)
pred = np.argmax(outputs.predictions, axis=1)
ifCorrect2 = (pred == outputs.label_ids)
WrongPred2 = np.where(~ifCorrect2)

In [ ]:
len(WrongPred2[0])/500

In [ ]:
outputs

In [ ]:
pred

In [ ]:
train_ds[:]['label']

In [ ]:


diff_corr1 = [] #model stage 1
diff_corr2 = [] #model stage 2
num_epoch = 100
for i in range(500):
    f1 = f'{data_set}_validation_img_2stage_e{str(num_epoch)}/{i}_test.png'
    im_array1 = np.asarray(Image.open(f1).convert('RGB'))
    
    f2 = f'{data_set}_train_img_e{str(num_epoch)}/{i}_train.png'
    im_array2 = np.asarray(Image.open(f2).convert('RGB'))
    cor1 = np.corrcoef(im_array1.flatten(), im_array2.flatten())[0,1]
    
    #print(cor1)
    
    f1 = f'{data_set}_train_img_2stage_e{str(num_epoch)}/{i}_train.png'
    im_array1 = np.asarray(Image.open(f1).convert('RGB'))
    
    f2 = f'{data_set}_validation_img_e{str(num_epoch)}/{i}_test.png'
    im_array2 = np.asarray(Image.open(f2).convert('RGB'))
    cor2 = np.corrcoef(im_array1.flatten(), im_array2.flatten())[0,1]
    
    #print(cor2)
    diff_corr2.append(cor1)
    diff_corr1.append(cor2)


In [ ]:
import matplotlib.pyplot as plt

plt.scatter(ifCorrect2, diff_corr2)

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

df2 = pd.DataFrame(diff_corr2)
df2['Group'] = ifCorrect2
df2['Group'][df2['Group'] == 0] = 'Wrong'
df2['Group'][df2['Group'] == 1] = 'Correct'
df2.columns = ['TAVAC','Group']

# fig, ax = plt.subplots()
# fig.set_size_inches(5, 5)
# my_pal = {"Wrong": "blue", "Correct": "#d1e231"}
# sns.boxplot(x="Group",y="TAVAC",data=df2,ax=ax,width=0.5, showfliers = False, palette=my_pal)
# #sns.violinplot(x="Group",y="TAVAC",data=df,ax=ax,width=0.5, showfliers = False, palette=my_pal)

# plt.ylabel('TAVAC',fontsize=font_size)
# plt.xticks(fontsize=font_size)
# plt.title('Prediction vs TAVAC', fontsize=font_size)

In [ ]:
import scipy

from scipy import stats

# Two-sample t-test
statistic, pvalue = stats.ttest_ind(df['TAVAC'][df['Group'] == 'Wrong'],df['TAVAC'][df['Group'] == 'Correct'])

print(f'Two-sample t-test: s = {statistic:5.3f}, p = {pvalue:5.5f}')

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(ifCorrect1, diff_corr1)

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

df1 = pd.DataFrame(diff_corr1)
df1['Group'] = ifCorrect1
df1['Group'][df1['Group'] == 0] = 'Wrong'
df1['Group'][df1['Group'] == 1] = 'Correct'
df1.columns = ['TAVAC','Group']

df = df1.append(df2)
fig, ax = plt.subplots()
fig.set_size_inches(5, 5)
my_pal = {"Wrong": "blue", "Correct": "#d1e231"}
sns.boxplot(x="Group",y="TAVAC",data=df,ax=ax,width=0.5, showfliers = False, palette=my_pal)
#sns.violinplot(x="Group",y="TAVAC",data=df,ax=ax,width=0.5, showfliers = False, palette=my_pal)

plt.ylabel('TAVAC',fontsize=font_size)
plt.title(data_set+' prediction vs TAVAC',fontsize=font_size)
plt.xticks(fontsize=font_size)

In [ ]:
df.shape

In [ ]:
import scipy

from scipy import stats

# Two-sample t-test
statistic, pvalue = stats.ttest_ind(df['TAVAC'][df['Group'] == 'Wrong'],df['TAVAC'][df['Group'] == 'Correct'])

print(f'Two-sample t-test: s = {statistic:5.3f}, p = {pvalue:5.5f}')

In [ ]:
id2label = {id:label for id, label in enumerate(train_ds.features['label'].names)}
# id2label is dictionnary. Enumerate fuction is used to iterate over the names of the class labels 
# in the 'label' feature of the'train_ds' dataset. For each label, the corresponding ID(index) is
# assigned as a key, and the labe itself is assiged as the value.

label2id = {label:id for id,label in id2label.items()}
# reverses the order of each key-value pair in the dictionary 

id2label

In [ ]:
train_ds[499]['img']


In [ ]:
id2label[val_ds[14]['label']]


In [ ]:
!nvidia-smi

In [ ]:
trainer.predict??